In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diabetes-data-set/diabetes.csv


In [2]:
!pip install pyspark

     |████████████████████████████████| 212.3 MB 14 kB/s s eta 0:00:01     |████████████████                | 106.5 MB 18.4 MB/s eta 0:00:06
     |████████████████████████████████| 198 kB 35.6 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=8c69a58cdc5121f09fa3bdec844b31c2140137775ef9557d3c9bbd37e3ca63bc
  Stored in directory: /root/.cache/pip/wheels/43/47/42/bc413c760cf9d3f7b46ab7cd6590e8c47ebfd19a7386cd4a57
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [7]:
df = spark.read.csv('/kaggle/input/diabetes-data-set/diabetes.csv', inferSchema=True, header=True)

In [8]:
df.take(5)

[Row(Pregnancies=6, Glucose=148, BloodPressure=72, SkinThickness=35, Insulin=0, BMI=33.6, DiabetesPedigreeFunction=0.627, Age=50, Outcome=1),
 Row(Pregnancies=1, Glucose=85, BloodPressure=66, SkinThickness=29, Insulin=0, BMI=26.6, DiabetesPedigreeFunction=0.351, Age=31, Outcome=0),
 Row(Pregnancies=8, Glucose=183, BloodPressure=64, SkinThickness=0, Insulin=0, BMI=23.3, DiabetesPedigreeFunction=0.672, Age=32, Outcome=1),
 Row(Pregnancies=1, Glucose=89, BloodPressure=66, SkinThickness=23, Insulin=94, BMI=28.1, DiabetesPedigreeFunction=0.167, Age=21, Outcome=0),
 Row(Pregnancies=0, Glucose=137, BloodPressure=40, SkinThickness=35, Insulin=168, BMI=43.1, DiabetesPedigreeFunction=2.288, Age=33, Outcome=1)]

In [9]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [10]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [11]:
print(df.count(), len(df.columns))

768 9


In [17]:
print('Total number of diabetic and non-diabetic:')
print(df.groupBy('Outcome').count().show())

Total number of diabetic and non-diabetic:
+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  268|
|      0|  500|
+-------+-----+

None


In [18]:
df.describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+
|summary|       Pregnancies|          Glucose|     BloodPressure|     SkinThickness|           Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+
|  count|               768|              768|               768|               768|               768|               768|                     768|               768|               768|
|   mean|3.8450520833333335|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.992578124999977|      0.4718763020833327|33.240885416666664|0.3489583333333333|
| stddev|  3.36957806269887|31.97261819513622|19.355807170644777|15.95

In [19]:
for col in df.columns:
    print(col + ': ', df[df[col].isNull()].count())

Pregnancies:  0
Glucose:  0
BloodPressure:  0
SkinThickness:  0
Insulin:  0
BMI:  0
DiabetesPedigreeFunction:  0
Age:  0
Outcome:  0


In [24]:
for col in df.columns:
    print('Correlation between Outcome column and ' + col + ' is', df.stat.corr('Outcome', col))

Correlation between Outcome column and Pregnancies is 0.22189815303398638
Correlation between Outcome column and Glucose is 0.46658139830687373
Correlation between Outcome column and BloodPressure is 0.06506835955033279
Correlation between Outcome column and SkinThickness is 0.07475223191831945
Correlation between Outcome column and Insulin is 0.13054795488404797
Correlation between Outcome column and BMI is 0.29269466264444527
Correlation between Outcome column and DiabetesPedigreeFunction is 0.17384406565296007
Correlation between Outcome column and Age is 0.23835598302719757
Correlation between Outcome column and Outcome is 1.0


In [27]:
# feature selection

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI'], outputCol='features')
output = assembler.transform(df)

In [28]:
output.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|[6.0,148.0,72.0,3...|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|[1.0,85.0,66.0,29...|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|[8.0,183.0,64.0,0...|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|[1.0,89.0,66.0,23...|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|[0.0,137.0,40.0,3...|
|          5|    116|           

In [29]:
output.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [30]:
from pyspark.ml.classification import LogisticRegression

final_data = output.select('features', 'Outcome')

In [31]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [32]:
train, test = final_data.randomSplit([0.8,0.2])
logistic_regression = LogisticRegression(labelCol='Outcome')
model = logistic_regression.fit(train)

In [36]:
summary = model.summary
summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|            Outcome|         prediction|
+-------+-------------------+-------------------+
|  count|                624|                624|
|   mean|0.34615384615384615| 0.2900641025641026|
| stddev| 0.4761246198853256|0.45415576597539287|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [37]:
#evaluate model

from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = model.evaluate(test)

In [38]:
predictions.predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(6,[0,1],[3.0,80.0])|      0|[4.66669058572734...|[0.99068426140547...|       0.0|
|(6,[0,1],[7.0,105...|      0|[3.15220134399777...|[0.95899537331218...|       0.0|
|(6,[1,5],[117.0,3...|      0|[0.68293481967584...|[0.66439340366345...|       0.0|
|(6,[1,5],[119.0,3...|      1|[0.73599366069450...|[0.67611915964175...|       0.0|
|[0.0,86.0,68.0,32...|      0|[2.38447287148618...|[0.91563559337428...|       0.0|
|[0.0,93.0,60.0,0....|      0|[2.02087974299681...|[0.88297194578976...|       0.0|
|[0.0,100.0,88.0,6...|      0|[1.13078392598783...|[0.75598354084238...|       0.0|
|[0.0,101.0,76.0,0...|      0|[1.84634480543307...|[0.86369736980903...|       0.0|
|[0.0,105.0,64.0,4...|      0|[1.14843684581726...|[0.75922528464356...|    

In [40]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='Outcome')
print('Model accuracy: ', evaluator.evaluate(model.transform(test)))

Model accuracy:  0.7788461538461534
